# Customer Support Chatbot with CERT Monitoring

A **production-grade customer support chatbot** for a telecommunications company, demonstrating real-world conversational AI with quality monitoring.

## Real-World Use Case

Telecom companies handle millions of customer interactions daily:
- Billing disputes and payment issues
- Service outages and technical problems
- Plan changes and upgrades
- Contract cancellations (churn risk)

This chatbot handles these scenarios with:
- **Intent Classification**: Automatically categorize customer issues
- **Sentiment Tracking**: Monitor customer frustration levels
- **Escalation Detection**: Identify when human intervention is needed
- **Response Quality**: Ensure consistent, helpful responses

## Architecture

```
Customer Message
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  Intent Classifier (GPT-4o)                                 │
│  → billing | technical | account | cancellation | general  │
└─────────────────────────────────────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  Sentiment Analyzer (GPT-4o)                                │
│  → positive | neutral | frustrated | angry                  │
└─────────────────────────────────────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  Response Generator (GPT-4o)                                │
│  Context-aware response with company policies               │
└─────────────────────────────────────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  CERT Dashboard                                             │
│  • Trace all LLM calls                                      │
│  • Evaluate response quality via LLM Judge                  │
│  • Monitor costs and latency                                │
│  • Track escalation patterns                                │
└─────────────────────────────────────────────────────────────┘
```

## CERT Integration

Every LLM call is traced to CERT Dashboard for:
- **Quality Evaluation**: LLM Judge scores response helpfulness
- **Cost Tracking**: Monitor spend per conversation/intent type
- **Performance**: Track latency for SLA compliance
- **Escalation Analysis**: Identify problematic conversation patterns

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q openai pydantic requests

print("Packages installed successfully!")

In [ ]:
import os
from getpass import getpass

# =============================================================
# API KEYS CONFIGURATION
# =============================================================

# CERT Dashboard Configuration
CERT_DASHBOARD_URL = "https://dashboard.cert-framework.com"

# Your CERT API Key (from dashboard.cert-framework.com/account)
if 'CERT_API_KEY' not in os.environ:
    os.environ['CERT_API_KEY'] = getpass('Enter your CERT API Key: ')

# OpenAI API Key (YOUR key - never shared with CERT)
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

CERT_API_KEY = os.environ['CERT_API_KEY']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

print(f"CERT Dashboard: {CERT_DASHBOARD_URL}")
print(f"CERT API Key: {CERT_API_KEY[:20]}...")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}... (stays local, never sent to CERT)")

## 2. CERT Tracer Setup

In [ ]:
import requests
import time
import uuid
from datetime import datetime
from typing import List, Optional, Dict, Any

class CERTTracer:
    """
    CERT Dashboard Tracer for LLM monitoring.
    
    Sends trace data to CERT Dashboard for:
    - Quality evaluation (LLM Judge)
    - Cost analysis
    - Performance monitoring
    - Compliance tracking
    """
    
    def __init__(self, dashboard_url: str, api_key: str, project_name: str = "default"):
        self.endpoint = f"{dashboard_url.rstrip('/')}/api/v1/traces"
        self.api_key = api_key
        self.project_name = project_name
        self.session_id = str(uuid.uuid4())[:8]
        self.traces_sent = 0
        self.total_tokens = 0
        self.total_cost_usd = 0.0
        
    def _get_headers(self) -> Dict[str, str]:
        return {
            "Content-Type": "application/json",
            "X-API-Key": self.api_key
        }
    
    def _estimate_cost(self, model: str, prompt_tokens: int, completion_tokens: int) -> float:
        """Estimate cost based on model pricing (as of 2024)."""
        pricing = {
            "gpt-4o": {"input": 2.50 / 1_000_000, "output": 10.00 / 1_000_000},
            "gpt-4o-mini": {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
            "gpt-4-turbo": {"input": 10.00 / 1_000_000, "output": 30.00 / 1_000_000},
        }
        
        if model in pricing:
            return (prompt_tokens * pricing[model]["input"] + 
                    completion_tokens * pricing[model]["output"])
        return 0.0
    
    def send_trace(self, trace_data: dict) -> bool:
        try:
            response = requests.post(
                self.endpoint,
                json={"traces": [trace_data]},
                headers=self._get_headers(),
                timeout=10
            )
            
            if response.status_code == 200:
                self.traces_sent += 1
                return True
            else:
                print(f"  [CERT] Error {response.status_code}: {response.text[:100]}")
                return False
                
        except Exception as e:
            print(f"  [CERT] Connection error: {e}")
            return False
    
    def trace_llm_call(
        self,
        provider: str,
        model: str,
        input_text: str,
        output_text: str,
        duration_ms: float,
        prompt_tokens: int = 0,
        completion_tokens: int = 0,
        metadata: Optional[Dict[str, Any]] = None
    ) -> dict:
        """Record an LLM call trace."""
        
        # Track totals
        self.total_tokens += prompt_tokens + completion_tokens
        self.total_cost_usd += self._estimate_cost(model, prompt_tokens, completion_tokens)
        
        trace = {
            "id": f"{self.session_id}-{str(uuid.uuid4())[:8]}",
            "provider": provider,
            "model": model,
            "input": input_text,
            "output": output_text,
            "promptTokens": prompt_tokens,
            "completionTokens": completion_tokens,
            "durationMs": round(duration_ms, 2),
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "metadata": {
                "project": self.project_name,
                "session_id": self.session_id,
                **(metadata or {})
            }
        }
        
        self.send_trace(trace)
        return trace
    
    def get_stats(self) -> Dict[str, Any]:
        return {
            "traces_sent": self.traces_sent,
            "total_tokens": self.total_tokens,
            "estimated_cost_usd": round(self.total_cost_usd, 4)
        }

# Initialize tracer
tracer = CERTTracer(
    dashboard_url=CERT_DASHBOARD_URL,
    api_key=CERT_API_KEY,
    project_name="telecom-customer-support"
)

print(f"CERT Tracer initialized")
print(f"  Session: {tracer.session_id}")
print(f"  Project: {tracer.project_name}")

## 3. Company Knowledge Base

Real customer support requires access to company policies, pricing, and procedures.

In [ ]:
# =============================================================
# TELECOM COMPANY KNOWLEDGE BASE
# =============================================================

COMPANY_NAME = "TelcoMax"

COMPANY_POLICIES = """
# TelcoMax Customer Service Policies

## Billing Policies
- Payment due date: 15th of each month
- Late payment fee: $10 after 5 days grace period
- Disputed charges: Must be reported within 60 days
- Refund processing: 5-7 business days to original payment method
- Bill credits: Agents can apply up to $50 credit without supervisor approval
- Credits over $50: Require supervisor approval (escalate)

## Service Plans
- Basic: $45/month - 5GB data, unlimited talk/text
- Standard: $65/month - 15GB data, unlimited talk/text, HBO Max
- Premium: $85/month - Unlimited data, unlimited talk/text, all streaming
- Family plans: 20% discount for 3+ lines

## Technical Support
- Service outages: Check status at status.telcomax.com
- Network issues: Basic troubleshooting required before technician dispatch
- Device issues: 14-day return policy, 1-year manufacturer warranty
- Technician visits: Free for service issues, $75 for customer-caused problems

## Account Management
- Plan changes: Effective immediately, prorated billing
- Contract cancellation: $150 early termination fee if within 12 months
- Account transfer: Requires identity verification and $25 transfer fee
- International roaming: Must be enabled before travel, $10/day pass available

## Escalation Triggers (MUST escalate to human agent)
- Customer mentions "lawyer" or "legal action"
- Customer requests to speak to supervisor more than once
- Billing dispute over $200
- Account security concerns (fraud, unauthorized access)
- Customer expresses intent to harm themselves or others
- Customer uses threatening language toward agent
"""

CUSTOMER_DATABASE = {
    "CUST-001": {
        "name": "John Smith",
        "plan": "Premium",
        "monthly_bill": 85.00,
        "account_since": "2019-03-15",
        "balance_due": 170.00,
        "last_payment": "2024-10-01",
        "phone_number": "555-0101",
        "devices": ["iPhone 15 Pro"],
        "notes": "Long-term customer, usually pays on time"
    },
    "CUST-002": {
        "name": "Maria Garcia",
        "plan": "Standard",
        "monthly_bill": 65.00,
        "account_since": "2022-08-20",
        "balance_due": 0.00,
        "last_payment": "2024-11-10",
        "phone_number": "555-0102",
        "devices": ["Samsung Galaxy S24"],
        "notes": "Enrolled in autopay"
    },
    "CUST-003": {
        "name": "Robert Chen",
        "plan": "Family (4 lines)",
        "monthly_bill": 220.00,
        "account_since": "2021-01-05",
        "balance_due": 440.00,
        "last_payment": "2024-09-15",
        "phone_number": "555-0103",
        "devices": ["iPhone 14", "iPhone 13", "Pixel 8", "iPhone SE"],
        "notes": "Payment issues last 2 months, offered payment plan previously"
    }
}

print(f"Knowledge base loaded for {COMPANY_NAME}")
print(f"  Customers in database: {len(CUSTOMER_DATABASE)}")

## 4. Customer Support Chatbot Components

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Literal
from enum import Enum
import json

# Initialize OpenAI client
client = OpenAI()

# =============================================================
# DATA MODELS
# =============================================================

class IntentType(str, Enum):
    BILLING = "billing"
    TECHNICAL = "technical"
    ACCOUNT = "account"
    CANCELLATION = "cancellation"
    GENERAL = "general"

class SentimentType(str, Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    FRUSTRATED = "frustrated"
    ANGRY = "angry"

class IntentClassification(BaseModel):
    intent: str = Field(description="The primary intent: billing, technical, account, cancellation, or general")
    confidence: float = Field(description="Confidence score 0-1")
    sub_intent: str = Field(description="More specific categorization")
    requires_account_lookup: bool = Field(description="Whether customer account info is needed")

class SentimentAnalysis(BaseModel):
    sentiment: str = Field(description="Current sentiment: positive, neutral, frustrated, or angry")
    intensity: float = Field(description="Intensity 0-1, where 1 is extreme")
    escalation_risk: bool = Field(description="Whether this conversation might need escalation")
    key_emotions: list[str] = Field(description="List of detected emotions")

class EscalationCheck(BaseModel):
    should_escalate: bool = Field(description="Whether to escalate to human agent")
    reason: str = Field(description="Reason for escalation decision")
    urgency: str = Field(description="low, medium, high, or critical")

print("Data models defined")

In [ ]:
# =============================================================
# INTENT CLASSIFIER
# =============================================================

def classify_intent(message: str, conversation_history: list) -> IntentClassification:
    """
    Classify the customer's intent from their message.
    Uses GPT-4o for accurate multi-class classification.
    """
    
    system_prompt = """You are an intent classifier for TelcoMax customer support.
    
Classify the customer message into one of these intents:
- billing: Payment issues, charges, refunds, billing disputes
- technical: Service outages, network issues, device problems, connectivity
- account: Plan changes, upgrades, account settings, personal info updates
- cancellation: Wants to cancel service, threatens to leave, competitor mentions
- general: Greetings, general questions, unclear intent

Return a JSON object with:
- intent: The primary intent category
- confidence: 0-1 confidence score
- sub_intent: More specific categorization (e.g., "late_fee_dispute", "slow_internet")
- requires_account_lookup: true if we need customer account details"""
    
    # Build context from conversation history
    context = ""
    if conversation_history:
        context = "Previous messages:\n" + "\n".join(
            [f"{m['role']}: {m['content'][:100]}..." for m in conversation_history[-3:]]
        )
    
    start_time = time.time()
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"{context}\n\nCurrent message: {message}"}
        ],
        response_format={"type": "json_object"},
        temperature=0.1,
        max_tokens=200
    )
    
    duration_ms = (time.time() - start_time) * 1000
    result = json.loads(response.choices[0].message.content)
    
    # Send trace to CERT
    tracer.trace_llm_call(
        provider="openai",
        model="gpt-4o",
        input_text=f"[Intent Classification] {message[:200]}",
        output_text=json.dumps(result),
        duration_ms=duration_ms,
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        metadata={
            "task": "intent_classification",
            "detected_intent": result.get("intent"),
            "confidence": result.get("confidence")
        }
    )
    
    return IntentClassification(**result)

print("Intent classifier ready")

In [ ]:
# =============================================================
# SENTIMENT ANALYZER
# =============================================================

def analyze_sentiment(message: str, conversation_history: list) -> SentimentAnalysis:
    """
    Analyze customer sentiment and escalation risk.
    Critical for identifying frustrated customers before they churn.
    """
    
    system_prompt = """You are a sentiment analyzer for customer support conversations.

Analyze the customer's emotional state and provide:
- sentiment: positive, neutral, frustrated, or angry
- intensity: 0-1 scale (0.9+ is extreme)
- escalation_risk: true if showing signs of needing human intervention
- key_emotions: list of specific emotions detected

Escalation risk indicators:
- Mentions of lawyers, lawsuits, BBB, FCC complaints
- Repeated frustration across multiple messages
- Threats to cancel or switch providers
- Personal attacks or profanity
- Expressions of feeling unheard or dismissed

Return JSON only."""
    
    # Include recent history for context
    history_context = ""
    if conversation_history:
        history_context = "Conversation context:\n" + "\n".join(
            [f"{m['role']}: {m['content'][:150]}" for m in conversation_history[-4:]]
        )
    
    start_time = time.time()
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"{history_context}\n\nAnalyze this message: {message}"}
        ],
        response_format={"type": "json_object"},
        temperature=0.1,
        max_tokens=200
    )
    
    duration_ms = (time.time() - start_time) * 1000
    result = json.loads(response.choices[0].message.content)
    
    # Send trace to CERT
    tracer.trace_llm_call(
        provider="openai",
        model="gpt-4o",
        input_text=f"[Sentiment Analysis] {message[:200]}",
        output_text=json.dumps(result),
        duration_ms=duration_ms,
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        metadata={
            "task": "sentiment_analysis",
            "sentiment": result.get("sentiment"),
            "escalation_risk": result.get("escalation_risk")
        }
    )
    
    return SentimentAnalysis(**result)

print("Sentiment analyzer ready")

In [ ]:
# =============================================================
# RESPONSE GENERATOR
# =============================================================

def generate_response(
    message: str,
    conversation_history: list,
    intent: IntentClassification,
    sentiment: SentimentAnalysis,
    customer_info: dict = None
) -> str:
    """
    Generate a helpful, empathetic response based on context.
    Uses company policies and customer data when available.
    """
    
    system_prompt = f"""You are a helpful customer support agent for {COMPANY_NAME}.

## Your Personality
- Professional but warm and empathetic
- Solution-oriented, proactive
- Clear and concise communication
- Acknowledge customer feelings before problem-solving

## Company Policies
{COMPANY_POLICIES}

## Current Context
- Customer Intent: {intent.intent} ({intent.sub_intent})
- Customer Sentiment: {sentiment.sentiment} (intensity: {sentiment.intensity})
- Escalation Risk: {sentiment.escalation_risk}

## Customer Information
{json.dumps(customer_info, indent=2) if customer_info else 'Not available - ask for account number if needed'}

## Response Guidelines
1. If sentiment is frustrated/angry: Start with empathy acknowledgment
2. Address the specific issue based on intent
3. Provide clear next steps or resolution
4. If you can apply a credit (up to $50), offer it proactively
5. Keep response concise (2-4 sentences unless complex issue)
6. If escalation is needed based on policies, say you'll connect them to a supervisor"""
    
    # Build messages array with history
    messages = [{"role": "system", "content": system_prompt}]
    
    for msg in conversation_history[-6:]:  # Last 6 messages for context
        messages.append(msg)
    
    messages.append({"role": "user", "content": message})
    
    start_time = time.time()
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.7,
        max_tokens=300
    )
    
    duration_ms = (time.time() - start_time) * 1000
    response_text = response.choices[0].message.content
    
    # Send trace to CERT (this is the main response to evaluate)
    tracer.trace_llm_call(
        provider="openai",
        model="gpt-4o",
        input_text=message,
        output_text=response_text,
        duration_ms=duration_ms,
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        metadata={
            "task": "response_generation",
            "intent": intent.intent,
            "sentiment": sentiment.sentiment,
            "customer_id": customer_info.get("id") if customer_info else None,
            "escalation_risk": sentiment.escalation_risk
        }
    )
    
    return response_text

print("Response generator ready")

In [ ]:
# =============================================================
# MAIN CHATBOT CLASS
# =============================================================

class CustomerSupportChatbot:
    """
    Complete customer support chatbot with CERT monitoring.
    """
    
    def __init__(self, customer_id: str = None):
        self.conversation_history = []
        self.customer_id = customer_id
        self.customer_info = CUSTOMER_DATABASE.get(customer_id) if customer_id else None
        self.turn_count = 0
        self.escalated = False
        
    def process_message(self, message: str) -> dict:
        """
        Process a customer message through the full pipeline.
        Returns response and analysis metadata.
        """
        self.turn_count += 1
        
        print(f"\n{'='*60}")
        print(f"Turn {self.turn_count}: Processing customer message...")
        print(f"{'='*60}")
        
        # Step 1: Classify intent
        print("\n[1/3] Classifying intent...")
        intent = classify_intent(message, self.conversation_history)
        print(f"  Intent: {intent.intent} ({intent.sub_intent})")
        print(f"  Confidence: {intent.confidence:.2f}")
        
        # Step 2: Analyze sentiment
        print("\n[2/3] Analyzing sentiment...")
        sentiment = analyze_sentiment(message, self.conversation_history)
        print(f"  Sentiment: {sentiment.sentiment} (intensity: {sentiment.intensity:.2f})")
        print(f"  Emotions: {', '.join(sentiment.key_emotions)}")
        print(f"  Escalation Risk: {sentiment.escalation_risk}")
        
        # Check for escalation
        if sentiment.escalation_risk and not self.escalated:
            print("\n  *** ESCALATION TRIGGERED ***")
            self.escalated = True
        
        # Step 3: Generate response
        print("\n[3/3] Generating response...")
        response = generate_response(
            message=message,
            conversation_history=self.conversation_history,
            intent=intent,
            sentiment=sentiment,
            customer_info=self.customer_info
        )
        
        # Update conversation history
        self.conversation_history.append({"role": "user", "content": message})
        self.conversation_history.append({"role": "assistant", "content": response})
        
        return {
            "response": response,
            "intent": intent.model_dump(),
            "sentiment": sentiment.model_dump(),
            "escalated": self.escalated,
            "turn": self.turn_count
        }

print("CustomerSupportChatbot class ready")

## 5. Test Scenario 1: Billing Dispute (Frustrated Customer)

In [ ]:
print("="*60)
print("SCENARIO 1: Billing Dispute - Frustrated Customer")
print("="*60)
print("\nCustomer: John Smith (CUST-001) - Premium plan, owes $170")
print("Situation: Charged late fee, upset about it")

# Create chatbot with customer context
chatbot1 = CustomerSupportChatbot(customer_id="CUST-001")

# Conversation flow
messages = [
    "I just looked at my bill and there's a $10 late fee. This is ridiculous! I've been a customer for 5 years and always pay on time. One time I'm a few days late and you charge me?",
    "My payment was only 3 days late because my bank had a system issue. This isn't fair.",
    "Fine, I appreciate the credit. But I'm also concerned about my bill being higher than usual. Can you explain why it's $170 instead of $85?"
]

In [ ]:
# First message
result1 = chatbot1.process_message(messages[0])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result1['response']}")

In [ ]:
# Second message
result2 = chatbot1.process_message(messages[1])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result2['response']}")

In [ ]:
# Third message
result3 = chatbot1.process_message(messages[2])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result3['response']}")

## 6. Test Scenario 2: Technical Issue + Escalation

In [ ]:
print("\n" + "="*60)
print("SCENARIO 2: Technical Issue Escalating to Cancellation")
print("="*60)
print("\nCustomer: Maria Garcia (CUST-002) - Standard plan")
print("Situation: Ongoing service issues, considering switching providers")

chatbot2 = CustomerSupportChatbot(customer_id="CUST-002")

messages2 = [
    "My internet has been dropping constantly for the past week. I work from home and this is affecting my job. I've already restarted my router multiple times.",
    "I've done all the troubleshooting. This is the third time I'm calling about this issue this month. Nothing is being fixed!",
    "You know what? I've had it. Verizon is offering me a better deal anyway. I want to cancel my service. I'm done with TelcoMax."
]

In [ ]:
# First message - technical issue
result2_1 = chatbot2.process_message(messages2[0])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result2_1['response']}")

In [ ]:
# Second message - escalating frustration
result2_2 = chatbot2.process_message(messages2[1])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result2_2['response']}")

In [ ]:
# Third message - cancellation threat
result2_3 = chatbot2.process_message(messages2[2])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result2_3['response']}")

## 7. Test Scenario 3: Payment Difficulty (Empathy Required)

In [ ]:
print("\n" + "="*60)
print("SCENARIO 3: Customer with Payment Difficulty")
print("="*60)
print("\nCustomer: Robert Chen (CUST-003) - Family plan, owes $440")
print("Situation: Financial hardship, behind on payments")

chatbot3 = CustomerSupportChatbot(customer_id="CUST-003")

messages3 = [
    "Hi, I'm calling about my account. I know I'm behind on payments. I lost my job two months ago and I'm doing my best to catch up.",
    "I can probably pay $100 now, but I need some time for the rest. I have four lines because my kids need phones for school. Please don't shut off our service.",
]

In [ ]:
# First message - explaining situation
result3_1 = chatbot3.process_message(messages3[0])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result3_1['response']}")

In [ ]:
# Second message - payment plan request
result3_2 = chatbot3.process_message(messages3[1])
print(f"\n{'─'*60}")
print(f"AGENT RESPONSE:\n{result3_2['response']}")

## 8. CERT Dashboard Summary

In [ ]:
stats = tracer.get_stats()

print("="*60)
print("CERT DASHBOARD - SESSION SUMMARY")
print("="*60)
print(f"\nSession ID: {tracer.session_id}")
print(f"Project: {tracer.project_name}")
print(f"\nMetrics:")
print(f"  Total traces sent: {stats['traces_sent']}")
print(f"  Total tokens used: {stats['total_tokens']:,}")
print(f"  Estimated cost: ${stats['estimated_cost_usd']:.4f}")
print(f"\nView your traces and run evaluations at:")
print(f"  {CERT_DASHBOARD_URL}/quality")
print(f"\nRecommended evaluations:")
print(f"  1. LLM Judge - Evaluate response helpfulness and accuracy")
print(f"  2. Review escalated conversations for quality assurance")
print(f"  3. Compare intent classification accuracy across scenarios")
print("="*60)

## 9. Interactive Mode

Try your own conversation with the chatbot!

In [ ]:
def interactive_chat():
    """Run an interactive chat session."""
    print("\n" + "="*60)
    print("INTERACTIVE CUSTOMER SUPPORT CHAT")
    print("="*60)
    print(f"\nWelcome to {COMPANY_NAME} Customer Support!")
    print("Type 'quit' to end the conversation.")
    print("Type 'stats' to see CERT metrics.")
    print("-"*60)
    
    chatbot = CustomerSupportChatbot(customer_id="CUST-001")
    
    while True:
        try:
            user_input = input("\nYou: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() == 'quit':
                print("\nThank you for contacting TelcoMax. Have a great day!")
                break
            
            if user_input.lower() == 'stats':
                stats = tracer.get_stats()
                print(f"\n[CERT Stats] Traces: {stats['traces_sent']}, "
                      f"Tokens: {stats['total_tokens']:,}, "
                      f"Cost: ${stats['estimated_cost_usd']:.4f}")
                continue
            
            result = chatbot.process_message(user_input)
            print(f"\nAgent: {result['response']}")
            
            if result['escalated']:
                print("\n[System: This conversation has been flagged for supervisor review]")
                
        except KeyboardInterrupt:
            print("\n\nSession ended.")
            break

# Uncomment to run interactive mode:
# interactive_chat()

---

## Summary

### What This Notebook Demonstrates

| Feature | Implementation |
|---------|----------------|
| **Multi-turn Conversations** | Maintains conversation history for context-aware responses |
| **Intent Classification** | GPT-4o classifies billing, technical, account, cancellation intents |
| **Sentiment Analysis** | Real-time emotional state tracking with escalation detection |
| **Company Policies** | Responses grounded in actual business rules |
| **Customer Context** | Personalizes responses based on account history |
| **Escalation Handling** | Automatic detection and routing of high-risk conversations |

### CERT Dashboard Integration

All LLM calls are traced with metadata for:
- **Quality Evaluation**: Use LLM Judge to score response helpfulness
- **Intent Accuracy**: Track classification performance
- **Sentiment Trends**: Monitor customer satisfaction
- **Cost Analysis**: Track spending by conversation type
- **Escalation Patterns**: Identify common escalation triggers

### Real-World Applications

This pattern applies to:
- Telecommunications support
- Banking and financial services
- E-commerce customer service
- Healthcare patient support
- SaaS product support